In [ ]:
def gaussian_elim(M, m):
    '''
    Inputs:
        M - a matrix of the form
                        1 0 0   ...   0 0 0
                        - - -    a    - - -
                        - - -   a^2   - - -
                                 .
                                 .
                                 .
                        - - - a^(d-1) - - -
                        - - -    b    - - -
                        - - -    ab   - - -
                                 .
                                 .
                                 .
                        - - -    b^2  - - -
                                 .
                                 .
                                 .
                        - a^(d-1)b^(m/d-1)-
                        - - -   norm  - - -
        m - the number of rows in the matrix M

    Returns the last row of the matrix M as a linear combination of the first m rows as a vector.
    '''
    I = identity_matrix(M.base_ring(), m+1)
    for j in range(m):
        for i in range(j+1, m+1):
            # Remove all entries of column j past the (j,j) entry.
            I.add_multiple_of_row(i, j, -1*M[i,j])
            M.add_multiple_of_row(i, j, -1*M[i,j])
        for k in range(j+1, m):
            if M[k,j+1]!=0:
                I.swap_rows(j+1, k)
                M.swap_rows(j+1, k)
                I.rescale_row(j+1, 1/M[j+1,j+1])
                M.rescale_row(j+1, 1/M[j+1,j+1])
    I *= -1
    return list(I.submatrix(m, 0, 1, m)[0])


def edit_matrix(mat, editor):
    '''
    Inputs:
        mat - an input matrix
        editor - function to modify each entry in the matrix

    Edits each entry of an input matrix and returns the edited matrix.
    '''
    unwind = list(map(list, list(mat)))
    edited = list(map(lambda vect: list(map(lambda ent: editor(ent), vect)) , unwind))
    return Matrix(edited)


def mult_by_elt(elt, F, Ex, Ex_to_F):
    '''
    Inputs:
        elt - an element in the field F
        F - a field over E
        Ex - a subfield of F over E
        Ex_to_F - an embedding from the subfield Ex into F
    
    Returns the matrix associated to multiplication by an element in F, acting on the vector space
    over a subfield Ex.
    '''
    m = F.absolute_degree()/Ex.absolute_degree()
    mat = []
    Fgen = F.0
    Exgen = Ex_to_F(Ex.0) #This is x, the generator of Ex over E
    d = Ex.relative_degree() #This is the degree of Ex over E
    basisMat = []

    # Convert to a new basis for F over E as x^i*Fgen^j.
    # Each element of this new basis is written as an element in terms of the old basis and becomes a row of 'basis'
    for i in range(d):
        for j in range(m):
            basisMat.append(( Exgen^i * Fgen^j ).list())

    #We will be using gaussian_elim on Matrix(basisMat) to write the last row in terms of our new basis
    basisMat.append(0.list())

    # We have Fgen^i ranging from 0 to m-1 a basis for F over Ex.
    # We will write the matrix of multiplication by 'elt' in terms of this basis
    for j in range(m):
        z = elt * Fgen^j
        # This sets the last row of basisMat to 'z' and rewrites it in terms of the basis x^i Fgen^j
        basisMat[-1]=z.list()
        matrow = gaussian_elim( Matrix(basisMat), m*d)
        #Matrow consists of the coefficients of z in terms of the basis x^i Fgen^j
        mat.append(matrow)

    # We use the expression of z in terms of x^i Fgen^j to write z in terms of Fgen^j with coeffs in Ex
    mult_by_elt = []
    for matrow in mat:
        rownewmat = []
        for j in range(m):
            rownewmat.append(0)
            for i in range(d):
                rownewmat[-1] += Exgen^i * Ex_to_F(matrow[i*m+j])
        mult_by_elt.append(rownewmat)
    return matrix(mult_by_elt).transpose()


def mult_by_pol(poly, F, Ex, Ex_to_F, E_pol, E_polT):
    '''
    Inputs:
        poly - a polynomial in x and y over F
        F - a NumberField
        Ex - a subfield of F
        Ex_to_F - an embedding of the subfield Ex into F
        E_pol - Multivariate Polynomial Ring in x and y over E
        E_pol_T - Univariate Polynomial Ring in T over a Fraction Field of the
                  Multivariate Polynomial Ring in x and y over E
    
    Expresses each element in F[x,y] as a sum of monomials in x, y with coefficients in F.
    For each coeff in F, we decompose it as a sum of the basis for F/E. This gives us a
    decomposition of our polynomial in terms of the same basis for F(x,y)/E(x,y).
    
    Returns the matrix associated to multiplication by a polynomial in x and y.
    '''
    M = None
    varname = E_polT.gens()[0]
    for mon in poly.monomials():
        coef = poly.monomial_coefficient(mon)
        Fcoef = F(coef)
        coefmat = mult_by_elt(Fcoef, F, Ex, Ex_to_F)
        
        # Now, the elements of the matrix coefmat lie in E(x). Replace x by T and shift the
        # coefficients to E_polT = (FractionField(E[x,y]))[T]. This is so that we can multiply
        # coefficients by Xs and Ys.
        newcoefmat = edit_matrix(coefmat, lambda ent: E_polT(ent.lift(var=varname)))
        if M == None:
            deg = coefmat.nrows()
            M = matrix(ring=E_polT, nrows=deg, ncols=deg)
        M += E_pol(mon)*newcoefmat
    return M


def Norm_Pol(z, F, Ex, Ex_to_F, E_pol, E_polT):
    '''
    Inputs:
        z - an element in F[x,y]
        F - a Number Field
        Ex - a subfield of F
        Ex_to_F - an embedding of Ex into F
        E_pol - Multivariate Polynomial Ring in x and y over E
        E_pol_T - Univariate Polynomial Ring in T over a Fraction Field of the
                  Multivariate Polynomial Ring in x and y over E

    Returns the determinant of the matrix of multiplication by the element z.
    '''
    M = mult_by_pol(z, F, Ex, Ex_to_F, E_pol, E_polT)
    return M.det()


def Norm(z, F, Ex, Ex_to_F = None, F_pol = None, E_pol=None, E_polT=None):
    '''
    Inputs:
        z - a tuple whose first entry is a list of numerators (each of which is a linear factor), and
            whose second entry is a list of denominators
        F - a field extension of Ex
        Ex - the base field
        Ex_to_F - an embedding of Ex into F
        F_pol - Multivariate Polynomial Ring in x and y over F
        E_pol - Multivariate Polynomial Ring in x and y over E
        E_polT - Univariate Polynomial Ring in T over a Fraction Field of the
                  Multivariate Polynomial Ring in x and y over E

    Returns the field norm of an element over an extension F/Ex.
    '''
    if z in F:
        return F(z).norm(Ex)
    if z in F_pol:
        return Norm_Pol(z, F, Ex, Ex_to_F, E_pol, E_polT)
    znum = z[0]
    zden = z[1]
    nm = 1
    for pol in znum:
        nm *= Norm_Pol(pol, F, Ex, Ex_to_F, E_pol, E_polT)

    # We use the fact that ell-th powers are trivial to calculate the norm of the denominator.
    for pol in zden:
        nm *= Norm_Pol(pol, F, Ex, Ex_to_F, E_pol, E_polT)^(ell-1)

    return nm


# Remove factors of T from the polynomial, since the reciprocity symbol (f/T)=0.
def star(p, E):
    '''
    Inputs:
        p - a polynomial in T defined over E
        E - a field

    If p(T) = a_n T^n + a_{n-1} T^{n-1} + ... + a_m T^m, returns the polynomial
    p*(T) = (a_m T^m)^{-1} p(T), as defined in
    
    Shmuel Rosset and John Tate. “A reciprocity law for K2-traces”. In: Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.
    '''
    if p in E:
        return 1
    T = p.args()[0]
    pcoeffs = p.coefficients(sparse=False)
    m = 0
    while pcoeffs[m]==0:
        m += 1
    q = p.shift(-m)
    q /= pcoeffs[m]
    return q


def c(p, E):
    '''
    Inputs:
        p - a polynomial in T defined over E
        E - a field

    If p(T) = a_n T^n + a_{n-1} T^{n-1} + ... + a_m T^m, returns c(p) = (-1)^n a_n, as defined in
    
    Shmuel Rosset and John Tate. “A reciprocity law forK2-traces”. In:Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.
    '''
    if p in E:
        return -p
    T = p.args()[0]
    return (-1)^p.degree(T) * p.coefficients()[-1]


def comp_tr(x, y, F, E, F_pol = None, E_pol = None):
    '''
    Inputs:
        x - an element of F
        y - an element of F
        F - an extension of E
        E - a base field
        F_pol - Multivariate Polynomial Ring in x and y over F
        E_pol - Multivariate Polynomial Ring in x and y over E

    Computes the transfer map which goes from K2(F) -> K2(E), together with the projection to the ell-torsion
    of the Brauer group of E. Implements the algorithm given in the Proposition of Section 3 in

    Shmuel Rosset and John Tate. “A reciprocity law for K2-traces”. In: Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.

    Returns the trace of x and y over the field extension F/E.
    '''

    # We consider two cases: (1) y is in F(x,y), or (2) y is in F.
    if F_pol == None:
        # Define f and g as polys in E[T] depending on x and y.
        R.<T> = E['T']
        g = x.minpoly(var='T').change_ring(E)
        Ex, Ex_to_F = F.subfield(x, 'Exgen')
        fbar = Norm(y, F, Ex, Ex_to_F)
        f = fbar.lift(var='T')
        constants = E

    else:
        E_polF = E_pol.fraction_field()
        F_polF = F_pol.fraction_field()
        E_polT.<T> = E_polF['T']
        F_polT = E_polT.base_extend(F_polF)

        # Get the subfield of F generated by x.
        g = x.minpoly(var='T')
        Ex, Ex_to_F = F.subfield(x, 'Exgen')
        fbar = E_polT(Norm(y, F, Ex, Ex_to_F, F_pol, E_pol, E_polT))

        # Define f as fbar reduced to be of minimal degree modulo g.
        _, f = fbar.quo_rem(g)
        constants = E_polF


    # Recursively compute the polynomials g_i; terminate when g_m is constant.
    glist = [g, f]
    glistar = [star(g, constants), star(f, constants)]
    while glist[-1] not in constants:
        _, newg = glistar[-2].quo_rem(glist[-1])
        glist.append(newg)
        glistar.append(star(newg, constants))
    tr = []

    # Recall that tr = -sum(c(star(g_i-1)), c(g_i)) by the Proposition in Section 3 of Rosset-Tate.
    # The inverse of the symbol (a, b) is (b, a) so we return sum (c(g_i), c(g_{i-1}^*)).
    for i in range(1, len(glist)):
        try:
            print("This string seems to fix the code sometimes")
            cg_star = c(glistar[i-1], constants)
            cg = c(glist[i], constants)
            tr.append((constants(cg), constants(cg_star)))
        except TypeError:
            print('SageMath sometimes throws the strange error "TypeError: sage.rings.integer.Integer object is not callable." Resolve this by adding a random print statement in the above for-loop and re-running relevant blocks of code.')
    return tr

In [ ]:
# These functions give us tP and tQ.

def tangent(pt, f, PolyRing):
    lamb = f.derivative(PolyRing(x)).subs({x:pt[0]})/(2*pt[1])
    const = lamb*pt[0] - pt[1]
    return PolyRing(y) - lamb*PolyRing(x) + const


def t_pt3(T, f, PolyRing):
    '''
    Inputs:
        T: a 3-torsion point
        f: a function in x

    Computes functions tP (with divisor 3[P]-3[O]) for 3-torsion.
    '''
    return ([tangent(T, f, PolyRing)], [])


def t_pt5(T, f, PolyRing):
    '''
    Inputs:
        T: a 5-torsion point
        f: a function in x

    Computes tP (with divisor 5[P]-5[O]) for 5-torsion.
    '''
    S = 2*T
    tang_to_T = tangent(T, f, PolyRing)
    tang_to_S = tangent(S, f, PolyRing)
    return ([tang_to_T, tang_to_T, PolyRing(x) - T[0] ],[tang_to_S])

In [ ]:
def act_on_pts(E, pt, sigma):
    '''
    Inputs:
        E: an EllipticCurve over a Number Field
        pt: a point on E
        sigma: an element in the Galois group of the field extension

    Returns the point sigma(pt) on the elliptic curve.
    '''
    if pt[2] == 0:
        return pt
    return E([sigma(pt[0]), sigma(pt[1]), 1])


def get_P(ELK, tor5, g):
    '''
    Inputs:
        tor5: 5-torsion points on an elliptic curve
        g: an element of the Galois group of the field extension

    Returns a generator of the 5-torsion subgroup of an elliptic curve.
    '''
    for pt in tor5:
        if (act_on_pts(ELK, pt, g) != pt) and (act_on_pts(ELK, pt, g) != 2*pt) and (act_on_pts(ELK, pt, g) != 3*pt) and (act_on_pts(ELK, pt, g) != 4*pt):
            P = pt
            return P

In [ ]:
def decompose(pt, P, Q, ell):
    '''
    Inputs:
        pt - a point on an elliptic curve
        P - first generator of ell-torsion of the elliptic curve
        Q - second generator of ell-torsion of the elliptic curve
        ell - the desired torsion

    Decomposes a point on an elliptic curve E in terms of the basis P, Q of the ell-torsion
    of E. If pt = i*P + j*Q, returns [i,j].
    '''
    for i in range(ell):
        for j in range(ell):
            if (pt== i*P + j*Q):
                return [i, j]


def get_rho(Gal, matGrp, P, Q, E):
    '''
    Inputs:
        Gal - Galois group of field extension L/K
        matGrp - a matrix subgroup of GL(2,ell)
        P - first generator of ell-torsion of the elliptic curve E
        Q - second generator of ell-torsion of the elliptic curve E
        E - an elliptic curve over a number field

    Returns a dictionary whose keys are Gal group elements and whose values
    are the elements of GL(2,ell) under the standard Gal representation with a choice of basis P, Q.
    '''
    rho = {}
    for h in Gal:
        hP = act_on_pts(E, P, h)
        hQ = act_on_pts(E, Q, h)
        col1 = decompose(hP, P, Q, 5)
        col2 = decompose(hQ, P, Q, 5)
        rho[h] = matGrp([ [ col1[0], col2[0] ], [ col1[1], col2[1] ] ])
    return rho


def make_small(entry):
    '''
    Inputs:
        entry - an element of Z/ell Z

    Returns the entry as an element of Z, with the representative chosen to be in (-ell/2, ell/2).
    '''
    entry = ZZ(entry)
    if entry > ell/2:
        entry = entry - ell
        return entry
    else:
        return entry


def N_PQ(a, b, rho_PQ):
    '''
    Inputs:
        a - an element of LK1 (modulo ell-th powers)
        b - an element of LK1 (modulo ell-th powers)
        rho_PQ - the representation which takes elements of our Galois group

    Returns N_PQ(a, b), with N_PQ being the map defined in Definition 2.9 of the paper,
    "Symbol Length in Brauer Groups of Elliptic Curves."
    '''
    nm = [1,1]
    # Note that we take the product over all h, not over all h^-1.
    for h in rho_PQ.keys():
        mat = matrix(rho_PQ[h])
        mat = edit_matrix(mat, make_small) # This ensures that the norm is 1, not just a 5th power.
        nm[0] *= h(a)^(mat[0][0])
        nm[0] *= h(b)^(mat[0][1])
        nm[1] *= h(a)^(mat[1][0])
        nm[1] *= h(b)^(mat[1][1])
    return nm

In [ ]:
# E: y^2 = x^3 + c
# Degree 4 extension, Galois group Z/4Z
c = 10
f = x^3 + c
ell = 5
K2.<zeta15> = CyclotomicField(15)
EK2 = EllipticCurve(K2,[0, c])

# This defines L as the division field of the ell torsion.
#     Labs is L as an absolute field,
#     LK2 is L as a relative field extension of K2.
Ltemp, phi = EK2.division_field(ell, 'Lgen', map=True)
Labs, Labs_to_Ltemp, Ltemp_to_Labs = Ltemp.optimized_representation()

ELabs = EllipticCurve(Labs,[0, c])

LK2.<nu> = Labs.relativize(Ltemp_to_Labs * phi)

ELK2 = EllipticCurve(LK2,[0, c])

LK2_to_Labs, Labs_to_LK2 = LK2.structure()
LK2_to_Ltemp = Labs_to_Ltemp * LK2_to_Labs
Ltemp_to_LK2 = Labs_to_LK2 * Ltemp_to_Labs

# We define K1 = Q(zeta_5) as a subfield of Labs
K1.<zeta5>, K1_to_Labs = Labs.subfield(Ltemp_to_Labs(phi(zeta15^3)))

# LK is L defined as a relative extension of K
LK1.<mu> = Labs.relativize(K1_to_Labs)
GK1 = LK1.automorphisms()
LK1_to_Labs, Labs_to_LK1 = LK1.structure()


LK1_to_LK2 = Labs_to_LK2 * LK1_to_Labs
LK2_to_LK2 = Labs_to_LK2 * LK2_to_Labs

ELK1 = EllipticCurve(LK1,[0, c])

# Now we define multivariable polynomial rings over Q, Kprime, K and L
# These will substitute for K(E) and L(E), since it is easier to calculate with these rings
Qpol.<x,y> = QQ[]
K1pol.<x,y> = K1[]
K2pol.<x,y> = Qpol.change_ring(base_ring=K2)
LK2pol.<x,y> = K2pol.change_ring(base_ring=LK2)

In [ ]:
g = GK1[2] # This is an order four element
O = ELK1([0, 1, 0])
tor5 = O.division_points(5)
P = get_P(ELK1, tor5, g)
Q = act_on_pts(ELK1, P, g)

G = SL(2,5)
rho_PQ = get_rho(GK1, G, P, Q, ELK1)
alpha, beta = N_PQ(mu + zeta5, mu, rho_PQ)
alphaLK2 = LK1_to_LK2(alpha)
P_LK2 = ELK2([LK1_to_LK2(P[0]), LK1_to_LK2(P[1]), 1])
t_2P = t_pt5(2*P_LK2, f, LK2pol)

# By the discussion in our paper, we first compute the corestriction of (alpha) from L(E) to K2(E).
cor_K2 = comp_tr(alphaLK2, t_2P, LK2, K2, LK2pol, K2pol)

# This consists of two symbols, the first of which is trivial. The non-trivial symbol is stored below.
print("SYMBOL LENGTH: ", len(cor_K2))
# We check that the second entry of the first symbol is trivial.
print(cor_K2[0][1]==1)
symb = cor_K2[1]

I = K2pol.ideal([y^2-x^3-10])
symb = (symb[0].numerator().reduce(I)/symb[0].denominator().reduce(I), symb[1].numerator().reduce(I)/symb[1].denominator().reduce(I) )

In [ ]:
# Save output to a text file.

file1 = open('example5.3_info.txt', 'w')
file1.write('K is: \n'+ str(K1)+ '\n\n\n')
file1.write('L is: \n'+ str(LK1)+ '\n\n\n')
file1.write('E is: \n'+ str(E)+ '\n\n\n')
file1.write('P is: \n'+ str(P)+ '\n\n\n')
file1.write('Q is: \n'+ str(Q)+ '\n\n\n')
file1.write('alpha is: \n'+ str(alpha)+ '\n\n\n')
file1.write('alpha is N_PQ of: \n'+ '(' + str(mu + zeta5)+ '  ,  ' + str(zeta5) + '\n\n\n')
file1.write('Gal(L/K) consists of the following automorphisms: \n'+ str(GK1)+ '\n\n\n')
file1.write('t_2P is: \n'+ str(t_2P)+ '\n\n\n')
file1.close()

file2 = open('example5.3_output.txt', 'w')
file2.write('Note: in our paper we cleared denominators for these symbols by multiplying the numerator and denominator of the first entry by 319831^10, and multiplying the numerator and denominator of our second entry by 319831^5.' + '\n\n\n')
file2.write('first entry of symbol is: \n' + str(symb[0]) + '\n\n\n')
file2.write('second entry of symbol is: \n' + str(symb[1]) + '\n\n\n')
file2.write('\n\n\n\n\n\n\n')
file2.write('numerator of first entry is: \n' + str(319831^10*symb[0].numerator())+ '\n\n\n')
file2.write('denominator of first entry is: \n' + str(319831^10*symb[0].denominator())+ '\n\n\n')
file2.write('numerator of second entry is: \n' + str(319831^5*symb[1].numerator())+ '\n\n\n')
file2.write('denominator of second entry is: \n' + str(319831^5*symb[1].denominator())+ '\n\n\n')
file2.close()